<h1 align="center"><font color="yellow">Bancos de dados vetoriais como memória para seus agentes de IA</font></h1>

<font color="yellow">Data Scientist.: PhD.Eddy Giusepe Chirinos Isidro</font>

Link de estudo:

* https://medium.com/sopmac-ai/vector-databases-as-memory-for-your-ai-agents-986288530443

# Evolução do banco de dados — de relacional para vetorial

A evolução do gerenciamento de dados viu uma mudança de bancos de dados relacionais (`SQL`), que são projetados para dados estruturados e dependem de esquemas fixos, para bancos de dados `NoSQL`, que oferecem mais flexibilidade na manipulação de dados não estruturados ou semiestruturados. `Os bancos de dados vetoriais representam o próximo passo nessa evolução`, fornecendo uma solução otimizada para gerenciar e consultar dados vetoriais de alta dimensão (ou seja, `Embeddings vetoriais`) , geralmente gerados por aplicativos de aprendizado de máquina e IA.

# Espaço de Alta Dimensão

`Embeddings vetoriais` são representações numéricas de objetos como: `palavras`, `imagens` ou outros pontos de dados em um espaço de alta dimensão.


<font color="orange">Um espaço de alta dimensão é um conceito matemático que representa um espaço com muitas dimensões, onde cada dimensão é um eixo separado ou feature dos dados. Em termos práticos, um espaço de alta dimensão é simplesmente uma maneira de descrever dados que possuem muitas features ou atributos.</font>


Eles são gerados usando modelos de aprendizado de máquina ou redes neurais pré-treinadas. Esses Embeddings capturam as relações e similaridades entre os objetos, tornando mais fácil para um computador entender e processar os dados.

## <font color="red">Vetores, Embeddings e Dimensões simplificadas</font>


Digamos que você tenha uma coleção de palavras e queira representá-las de uma forma que um computador possa entender e processar. Uma maneira de fazer isso é usando algo chamado `"embeddings"`.


Pense nos Embeddings como uma forma de transformar palavras em pontos em um mapa. Cada palavra tem seu próprio lugar no mapa, e palavras similares ficam próximas umas das outras, enquanto palavras diferentes ficam distantes. Este `“mapa”` é como uma grade, mas com mais do que apenas duas direções (`cima/baixo` e `esquerda/direita`).


As `“direções”` neste mapa são chamadas de `dimensões`. Cada dimensão é como uma característica ou característica diferente de uma palavra. `Por exemplo:` uma dimensão pode representar como é a palavra `"feliz"`, enquanto outra dimensão pode representar se é um animal ou não. <font color="yellow">Quanto mais dimensões tivermos, mais características poderemos capturar sobre cada palavra.</font>

Um `“vetor”` é como um conjunto de instruções que informa como chegar à localização de uma palavra no mapa. Ele contém números para cada dimensão que o ajudam a encontrar o local exato da palavra. Quando falamos de `“Embeddings de vetores”`, estamos falando desses conjuntos de números que representam a localização das palavras em nosso mapa multidimensional.


Então, em termos simples, os `Embeddings` são uma forma de transformar palavras em pontos em um mapa com muitas direções (dimensões), e os vetores são os conjuntos de números que nos ajudam a encontrar a localização de cada palavra nesse mapa.

# Bancos de dados vetoriais


Bancos de dados vetoriais, também conhecidos como `bancos de dados de pesquisa de similaridade` ou `bancos de dados de pesquisa do vizinho mais próximo`, são bancos de dados especializados projetados para armazenar e consultar Embeddings de vetores de forma eficiente. Eles permitem que você execute operações como encontrar os itens mais similares a um determinado vetor ou pesquisar itens que atendam a critérios de similaridade específicos.

Os bancos de dados tradicionais não são otimizados para essas tarefas, e é por isso que os `bancos de dados vetoriais` se tornaram cada vez mais populares.

# Como integrar bancos de dados vetoriais usando Python

Agora que definimos alguma teoria por trás deste tópico, vamos fazer a transição para a aplicação prática de `bancos de dados vetoriais` com `Pinecone`, `Chroma` e `LangChain` — todos usando `Embeddings de vetores OpenAI`.

## <font color="pink">API PINECONE</font>

Este código demonstra como usar o `Pinecone` e o `OpenAI` para realizar uma pesquisa de similaridade em um conjunto de documentos usando Embeddings da OpenAI.


```
!pip install pinecone-client openai
```

Link para mais detalhes: [pinecone.io](https://www.pinecone.io/).

In [13]:
# Isto é quando usas o arquivo .env:
import openai 
from dotenv import load_dotenv
import os
print('Carregando a minha chave Key: ', load_dotenv())
Eddy_API_KEY_OpenAI = os.environ['OPENAI_API_KEY']  
Eddy_API_KEY_Cohere = os.environ["COHERE_API_KEY"]
Eddy_API_KEY_HuggingFace = os.environ["HUGGINGFACEHUB_API_TOKEN"]
Eddy_API_KEY_SerpApi = os.environ["SERPAPI_API_KEY"]
Eddy_API_KEY_WolframAlpha = os.environ["WOLFRAM_ALPHA_APPID"]

Eddy_API_KEY_pinecone = os.environ["PINECONE_API_KEY"]


Carregando a minha chave Key:  True


## <font color="pink">CHROMA — Opção de armazenamento efêmero</font>

O código a seguir demonstra como usar `ChromaDB` e `OpenAI` para realizar uma pesquisa de similaridade em um conjunto de documentos.

```
!pip install chromadb openai
```

In [3]:
import os
import chromadb
import openai

OPENAI_API_KEY = Eddy_API_KEY_OpenAI


client = chromadb.Client()
openai.api_key = OPENAI_API_KEY

Using embedded DuckDB without persistence: data will be transient


In [4]:
def complete(prompt):
    res = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                       messages=[{"role": "user", "content": prompt}]
                                      )
    
    return res['choices'][0]['message']['content'].strip()

def get_ada_embedding(text):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input=text, model="text-embedding-ada-002")["data"][0]["embedding"]


In [5]:
CHROMA_TABLE_NAME = "chroma-openai"

from chromadb.utils import embedding_functions

openai_ef = embedding_functions.OpenAIEmbeddingFunction(api_key=OPENAI_API_KEY,
                                                        model_name="text-embedding-ada-002"
                                                       )

collection = client.create_collection(CHROMA_TABLE_NAME, embedding_function=openai_ef)


texts = [
        "Agentes de AI como funcionários virtuais são o futuro.",
        "Bancos de dados vetoriais são o futuro.",
        "AGI não está aqui... ainda."
    ]

for loopIndex, text in enumerate(texts, start=1):
  collection.add(embeddings=get_ada_embedding(text), metadatas=[{"text": text}], ids=["test-openai-"+str(loopIndex)]) 
    


In [8]:
query_text = "Os agentes de AI são o futuro?"


results = collection.query(query_embeddings=get_ada_embedding(query_text),
                           n_results=3
                          )
results

{'ids': [['test-openai-1', 'test-openai-2', 'test-openai-3']],
 'embeddings': None,
 'documents': [[None, None, None]],
 'metadatas': [[{'text': 'Agentes de AI como funcionários virtuais são o futuro.'},
   {'text': 'Bancos de dados vetoriais são o futuro.'},
   {'text': 'AGI não está aqui... ainda.'}]],
 'distances': [[0.14552298188209534, 0.34075218439102173, 0.4094182848930359]]}

In [9]:
import json
print(json.dumps(results, indent=2))


{
  "ids": [
    [
      "test-openai-1",
      "test-openai-2",
      "test-openai-3"
    ]
  ],
  "embeddings": null,
  "documents": [
    [
      null,
      null,
      null
    ]
  ],
  "metadatas": [
    [
      {
        "text": "Agentes de AI como funcion\u00e1rios virtuais s\u00e3o o futuro."
      },
      {
        "text": "Bancos de dados vetoriais s\u00e3o o futuro."
      },
      {
        "text": "AGI n\u00e3o est\u00e1 aqui... ainda."
      }
    ]
  ],
  "distances": [
    [
      0.14552298188209534,
      0.34075218439102173,
      0.4094182848930359
    ]
  ]
}


In [10]:
client.delete_collection(name="chroma-openai")

1. Instale as bibliotecas necessárias: `chromadb` e `openai`.

2. Configure a chave de API OpenAI.

3. Crie uma instância do `cliente ChromaDB`.

4. Defina uma função `complete` para gerar uma resposta do modelo `GPT-3.5-turbo`, dado um `prompt`.

5. Defina uma função `get_ada_embedding` para obter Embeddings para texto de entrada usando o modelo `"text-embedding-ada-002"` da `OpenAI`.

6. Defina o nome da `tabela ChromaDB`.

7. Importe os utilitários necessários da `biblioteca chromadb`.

8. Crie um objeto de função de Embeddings OpenAI usando a chave de API OpenAI e o modelo `“text-embedding-ada-002”`.

9. Crie uma `coleção ChromaDB` com o nome da tabela e a função de Embedding especificados.

10. Defina uma `lista de textos` para servir como documentos a serem `indexados e pesquisados`.

11. Percorra os textos, obtenha Embeddings usando a função `get_ada_embedding` e adicione-os à coleção `ChromaDB` junto com metadados (`text content`) e um identificador exclusivo.

12. Defina o texto da query para a pesquisa de similaridade.

13. Realize uma pesquisa de similaridade na `coleção ChromaDB` usando os embeddings obtidos do texto da consulta e `recupere os 3 principais resultados mais similares`.

14. Imprima os resultados da pesquisa em uma representação `JSON` formatada.

15. Exclua a coleção ChromaDB.


<font color="orange">Em resumo, o código acima demonstra como usar `ChromaDB` e `OpenAI` para realizar uma pesquisa de similaridade em um conjunto de documentos, obtendo embeddings do modelo `OpenAI` `“text-embedding-ada-002”` e usando ChromaDB para armazenar e consultar esses embeddings.</font>

## <font color="pink">LANGCHAIN ​​VectorStore</font>

O código a seguir instala as bibliotecas necessárias e demonstra como usar dois `diferentes vector stores` (`Chroma` e `Pinecone`) para realizar pesquisas de similaridade em um conjunto de documentos usando os `Embeddings da OpenAI`.

```
!pip install pinecone-client chromadb openai langchain tiktoken
```

In [14]:
PINECONE_API_KEY = Eddy_API_KEY_pinecone
#PINECONE_ENV = "YOUR_PINECONE_ENVIRONMENT"
OPENAI_API_KEY = Eddy_API_KEY_OpenAI

In [15]:
import openai
openai.api_key = OPENAI_API_KEY


def get_ada_embedding(text):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input=text, model="text-embedding-ada-002")["data"][0]["embedding"]


In [16]:
texts = [
        "Agentes de AI como funcionários virtuais são o futuro.",
        "Bancos de dados vetoriais são o futuro.",
        "AGI não está aqui... ainda."
    ]


In [17]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

CHROMA_TABLE_NAME = "YOUR_CHROMA_TABLE_NAME"

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

query = "Qual é o futuro?"


docsearch = Chroma.from_texts(texts, embeddings)
docs = docsearch.similarity_search(query, k=2)
print(docs)

Using embedded DuckDB without persistence: data will be transient


[Document(page_content='Agentes de AI como funcionários virtuais são o futuro.', metadata={}), Document(page_content='Bancos de dados vetoriais são o futuro.', metadata={})]


In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone


PINECONE_TABLE_NAME = "YOUR_PINECONE_INDEX_NAME"

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

query = "Qual é o futuro?"

# docsearch = Pinecone.from_texts(texts, embeddings, index_name=PINECONE_TABLE_NAME)
# docs = docsearch.similarity_search(query, k=2)
# print(docs)

docsearch = Pinecone.from_existing_index(PINECONE_TABLE_NAME, embeddings)
docs = docsearch.similarity_search(query, k=2)
print(docs)